# **[Задание 2](https://github.com/pacifikus/itmo_ml_for_science_course/blob/main/HW/hw_2.md)**

# **Библиотеки**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline

# **Clear up**

In [ ]:
!kaggle datasets download abdulszz/spotify-most-streamed-songs

with zipfile.ZipFile('spotify-most-streamed-songs.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

Dataset URL: https://www.kaggle.com/datasets/abdulszz/spotify-most-streamed-songs
License(s): CC0-1.0
spotify-most-streamed-songs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
df = pd.read_csv('Spotify Most Streamed Songs.csv')
df.head(5)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,cover_url
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,B,Major,80,89,83,31,0,8,4,Not Found
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,C#,Major,71,61,74,7,0,10,4,https://i.scdn.co/image/ab67616d0000b2730656d5...
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,F,Major,51,32,53,17,0,31,6,https://i.scdn.co/image/ab67616d0000b273e85259...
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,A,Major,55,58,72,11,0,11,15,https://i.scdn.co/image/ab67616d0000b273e787cf...
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,A,Minor,65,23,80,14,63,11,6,https://i.scdn.co/image/ab67616d0000b273ab5c9c...


In [ ]:
df['streams'] = pd.to_numeric(df['streams'].str.replace(',', ''), errors='coerce')
print(f'Всего нечисловых значений: {df.streams.isna().sum()}')

Всего нечисловых значений: 1


In [ ]:
df = df.dropna(subset=['streams'])
print(f'Всего нечисловых значений: {df.streams.isna().sum()}')

Всего нечисловых значений: 0


In [ ]:
df['in_deezer_playlists'] = pd.to_numeric(df['in_deezer_playlists'].str.replace(',', '.'), errors='coerce')

In [ ]:
df['in_shazam_charts'] = pd.to_numeric(df['in_shazam_charts'].str.replace(',', '.'), errors='coerce')
df = df.dropna(subset=['in_shazam_charts'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 902 entries, 0 to 952
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            902 non-null    object 
 1   artist(s)_name        902 non-null    object 
 2   artist_count          902 non-null    int64  
 3   released_year         902 non-null    int64  
 4   released_month        902 non-null    int64  
 5   released_day          902 non-null    int64  
 6   in_spotify_playlists  902 non-null    int64  
 7   in_spotify_charts     902 non-null    int64  
 8   streams               902 non-null    float64
 9   in_apple_playlists    902 non-null    int64  
 10  in_apple_charts       902 non-null    int64  
 11  in_deezer_playlists   902 non-null    float64
 12  in_deezer_charts      902 non-null    int64  
 13  in_shazam_charts      902 non-null    float64
 14  bpm                   902 non-null    int64  
 15  key                   816 no

In [ ]:
df['artist(s)_name'] = df['artist(s)_name'].str.split(
    r'& |, ').explode('artist(s)_name')

In [ ]:
cat_cols = df.select_dtypes(include='object')
cat_cols.head(5)

,track_name,artist(s)_name,key,mode,cover_url
0,Seven (feat. Latto) (Explicit Ver.),Latto,B,Major,Not Found
1,LALA,Jung Kook,C#,Major,https://i.scdn.co/image/ab67616d0000b2730656d5...
2,vampire,Myke Towers,F,Major,https://i.scdn.co/image/ab67616d0000b273e85259...
3,Cruel Summer,Olivia Rodrigo,A,Major,https://i.scdn.co/image/ab67616d0000b273e787cf...
4,WHERE SHE GOES,Taylor Swift,A,Minor,https://i.scdn.co/image/ab67616d0000b273ab5c9c...


In [ ]:
cat_cols.shape

(902, 5)

In [ ]:
cat_cols.describe()

,track_name,artist(s)_name,key,mode,cover_url
count,902,902,816,902,902
unique,892,476,11,2,501
top,Let It Snow! Let It Snow! Let It Snow!,The Weeknd,C#,Major,Not Found
freq,2,31,115,521,219


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 902 entries, 0 to 952
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            902 non-null    object 
 1   artist(s)_name        902 non-null    object 
 2   artist_count          902 non-null    int64  
 3   released_year         902 non-null    int64  
 4   released_month        902 non-null    int64  
 5   released_day          902 non-null    int64  
 6   in_spotify_playlists  902 non-null    int64  
 7   in_spotify_charts     902 non-null    int64  
 8   streams               902 non-null    float64
 9   in_apple_playlists    902 non-null    int64  
 10  in_apple_charts       902 non-null    int64  
 11  in_deezer_playlists   902 non-null    float64
 12  in_deezer_charts      902 non-null    int64  
 13  in_shazam_charts      902 non-null    float64
 14  bpm                   902 non-null    int64  
 15  key                   816 no

# **Train/test split**

In [ ]:
np.random.seed(42)

In [ ]:
cat_cols = df.select_dtypes(include='object')
cat_cols.describe()

,track_name,artist(s)_name,key,mode,cover_url
count,902,902,816,902,902
unique,892,476,11,2,501
top,Let It Snow! Let It Snow! Let It Snow!,The Weeknd,C#,Major,Not Found
freq,2,31,115,521,219


Убираем из данных колонки:
 - `track_name`
 - `artist(s)_name`
 - `cover_url`

так как количество уникальных категорий соизмеримо с количеством наблюдений (по этим фичам недостаточная выборка).

Колонки `key`, `mode` будем кодировать с помощью `One-Hot-Encoding`, так как на них нет отношения порядка.

In [ ]:
encoded_columns = ['key', 'mode']

In [ ]:
X, y = df.drop(
    [
        'artist(s)_name',
        'track_name',
        'cover_url',
        'streams',  # целевое значение
    ], axis=1), df['streams']

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.3, random_state=42)  # Фиксируем random_state

In [ ]:
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((631, 21), (271, 21), (631,), (271,))

# **Dummy predictions**

In [ ]:
dummy = DummyRegressor(strategy='mean')
dummy.fit(train_X, train_y)

DummyRegressor()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from typing import Dict


def calculate_metrics(predictor) -> Dict[str, float]:
    return {
        'MSE': mean_squared_error(test_y, predictor.predict(test_X)),
        'MAPE': mean_absolute_percentage_error(test_y, predictor.predict(test_X)),
        'R2': r2_score(test_y, predictor.predict(test_X))
    }

In [ ]:
for name, metric in calculate_metrics(dummy).items():
    print(f'{name}: {metric}')

MSE: 2.5807544583401706e+17
MAPE: 2.3686679178309307
R2: -0.0017399091513614628


Получили бейзлайн (стоит учесть, что `R2` для идеального константного регрессора равен 0)

# **Baseline model**

Будем использовать `RobustScaler` для препроцессинга, чтобы нивелировать выбросы в обучающей выборке.

In [ ]:
baseline: Dict[str, Pipeline] = dict(
    linear=Pipeline([
        ('Preprocessor', ColumnTransformer([
            ('Scaler', RobustScaler(), list(
                set(train_X.columns).difference(encoded_columns))),
            ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'), encoded_columns)
        ])),
        ('Regressor', LinearRegression())
    ]),
    tree=Pipeline([
        ('Preprocessor', ColumnTransformer([
            ('TargetEncoder', TargetEncoder(target_type='continuous', shuffle=False), encoded_columns)
        ])),
        ('Regressor', DecisionTreeRegressor())
    ]),
    knn=Pipeline([
        ('Preprocessor', ColumnTransformer([
            ('Scaler', RobustScaler(), list(
                set(train_X.columns).difference(encoded_columns))),
            ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'), encoded_columns)
        ])),
        ('Regressor', KNeighborsRegressor())
    ]),
)

> Для модели дерева `Target Encoding` является более подходящим методом кодирования для деревьев решений благодаря своей способности сохранять информацию о целевой переменной и избегать увеличения размерности данных.



# **Сравнение моделей**

In [ ]:
for name, estimator in baseline.items():
    estimator.fit(train_X, train_y)
    print(f'________{name}________')
    for metric, value in calculate_metrics(estimator).items():
        print(f'{metric}: {value}')

________linear________
MSE: 8.646130586213563e+16
MAPE: 1.0119777396532406
R2: 0.6643937186680334
________tree________
MSE: 2.850396826649813e+17
MAPE: 1.9013751837662711
R2: -0.10640369096173319
________knn________
MSE: 1.022643951102742e+17
MAPE: 0.9108973923926973
R2: 0.6030527990133882


> **MSE:** меньшее значение MSE указывает на лучшую производительность модели. \
**MAPE:** аналогично, более низкое значение MAPE указывает на лучшую точность предсказаний в процентном выражении. \
**R²:** этот коэффициент показывает, какую долю дисперсии целевой переменной объясняет модель.

# Вывод


Из данных бейзлайновых моделей простого семейства по всем метрикам побеждает **линейная регрессия**, так как она имеет наименьшие значения MSE, неплохой результат по MAPE, а также значение R² свидетельствует о том, что она объясняет большую часть вариации в данных.